In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
!pip install torch_geometric
!pip install pillow

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/util

In [19]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from torch_geometric.utils.convert import from_networkx
import cv2

def draw_graph(g):
  X_i = []
  Y_i = []
  for node in g.nodes():
    X_i.append(g.nodes[node]['x'][0])
    Y_i.append(g.nodes[node]['x'][1])
  X_s = [[],[]]
  Y_s = [[],[]]
  dic = g.adjacency()
  for key in dic:
    for nei in key[1]:
      X_s[0].append(g.nodes[key[0]]['x'][0])
      X_s[1].append(g.nodes[nei]['x'][0])
      Y_s[0].append(g.nodes[key[0]]['x'][1])
      Y_s[1].append(g.nodes[nei]['x'][1])
  plt.plot(X_s,Y_s)
  plt.scatter(X_i,Y_i)
  plt.xlim(0,1)
  plt.ylim(0,1)
  plt.show()
  return


def check_if_adj(colors,p1,p2,color_adj):
    h,w = colors.shape
    x1,y1 =p1
    x2, y2 = p2
    x1 = int(x1)
    x2 = int(x2)
    y1 = int(y1)
    y2 = int(y2)
    c1 = colors[h-y1,x1]
    c2 = colors[h-y2,x2]
    if c1 == 0 or c2 == 0:
        return False

    if c2 in color_adj[int(c1)]:
        return True

def check_if_equal(p1,p2,font):
    x1,y1 = p1
    x2,y2 = p2
    if ((x1-x2)**2) + ((y1-y2)**2) <= font**2:
        return True
    return False

def find_font(h,w,img):
    S = np.zeros((h,w))
    for r in range(h):
        if img[r,0] == 0:
            S[r,0] = 1
        else:
            S[r,0] = 0
    for c in range(w):
        if img[0,c] == 0:
            S[0,c] = 1
        else:
            S[0,c] = 0



    for i in range(1,h):
        for j in range(1,w):
            if img[i,j] == 0:
                S[i,j] = min(S[i,j-1],S[i-1,j],S[i-1,j-1])+1
            else:
                S[i,j] = 0
    return np.max(S)




def generate_points(image_path):

    img = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    _, rect = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    X_s = []
    Y_s = []

    rect = np.array(img)
    h, w, = rect.shape
    for y in range(h):
        for x in range(w):
            if rect[y,x] < 128:
                X_s.append(x)
                Y_s.append(h - y)




    colors = np.zeros((h,w))
    X = np.asarray(X_s)
    Y = np.asarray(Y_s)
    x_min = np.min(X)
    y_min = np.min(Y)

    ind_x = []
    ind_y = []
    for i in range(0,7):
        for j in range(0,7):
            ind_x.append(((w*j)/6))
            ind_y.append((h*i)/6)

    categ_x = [[] for i in range(len(ind_x))]
    categ_y = [[] for i in range(len(ind_y))]
    ind_x = np.asarray(ind_x)
    ind_y = np.asarray(ind_y)
    for i in range(len(X)):
        x1 = X[i]
        y1 = Y[i]
        temp_x = ind_x - x1
        temp_y = ind_y - y1
        dist = np.square(temp_x) + np.square(temp_y)
        min_ind = np.argmin(dist)
        categ_x[min_ind].append(x1)
        categ_y[min_ind].append(y1)

    categ_x = [i for i in categ_x if i != []]
    categ_y = [i for i in categ_y if i != []]

    fin_x = []
    fin_y = []

    for i in range(len(categ_x)):
        mean_x = sum(categ_x[i])/len(categ_x[i])
        mean_y = sum(categ_y[i])/len(categ_y[i])

        if rect[h-int(mean_y),int(mean_x)] != 0:
            p = []
            for j in range(len(categ_x[i])):
                p.append((categ_x[i][j],categ_y[i][j]))
            p.sort()
            fin_x.append(p[int(len(categ_x[i])/2)][0])
            fin_y.append(p[int(len(categ_x[i])/2)][1])
        else:
            fin_x.append(mean_x)
            fin_y.append(mean_y)


    for i in range(len(categ_x)):
        for j in range(len(categ_x[i])):
            colors[h-categ_y[i][j],categ_x[i][j]] = i+1

    return fin_x,fin_y,colors





def around_ones(mat,color_adj):
    center = mat[1,1]
    l = [mat[0,0],mat[0,1],mat[0,2],mat[1,2],mat[2,2],mat[2,1],mat[2,0],mat[1,0]]
    for i in l:
        if l != 0:
            color_adj[int(center)].add(i)





def graph_making(img_name):
    fin_x, fin_y,colors = generate_points(img_name)


    color_adj = [set() for i in range(len(fin_x)+1)]


    #collor differnt points

    h,w = colors.shape
    for i in range(1,h-1):
        for j in range(1,w-1):
            if colors[i,j] != 0:
                around_ones(colors[i-1:i+2,j-1:j+2],color_adj)


    landmarks = []

    for i in range(len(fin_x)):
        landmarks.append((fin_x[i],fin_y[i]))

    pair_names = [0]*len(fin_x)
    point_named = [-1]*(len(fin_x)+1)

    name_count = 1
    # plt.imshow(colors)
    # plt.show()
    #landmarks.sort(reverse=True)
    adj_list = dict()
    for i in range(len(landmarks)):
        if pair_names[i] == 0:
            pair_names[i] = name_count
            adj_list[name_count] = set()
            name_count += 1

        for j in range(i+1, len(landmarks)):
            if check_if_adj(colors,landmarks[i],landmarks[j],color_adj):
                if pair_names[j] == 0:
                    pair_names[j] = name_count
                    name_count += 1
                if pair_names[j] not in adj_list.keys():
                    adj_list[pair_names[j]] = set()
                adj_list[pair_names[i]].add(pair_names[j])
                adj_list[pair_names[j]].add(pair_names[i])
    min_x = 10**9
    max_x =0
    min_y =10**9
    max_y = 0

    for i in range(len(fin_x)):
        point_named[pair_names[i]] = landmarks[i]
        x,y = landmarks[i]
        if x > max_x:
          max_x = x
        if y > max_y:
          max_y = y
        if x < min_x:
          min_x = x
        if y < min_y:
          min_y = y


    graph = nx.Graph()
    for key in adj_list.keys():
      graph.add_nodes_from([key])
      graph.nodes[key]['x'] = [(point_named[key][0]-min_x)/(max_x-min_x),(point_named[key][1]-min_y)/(max_y-min_y)]
    for key in adj_list.keys():
      for nei in adj_list[key]:
        graph.add_edge(key,nei)

    # draw_graph(graph)
    labels = nx.get_node_attributes(graph, 'x')
    # nx.draw(graph ,labels=labels,with_labels=True, node_color='lightblue', node_size=100, font_size=12, font_color='black', edge_color='gray')
    # plt.show()
    return from_networkx(graph)





In [20]:

print(graph_making("/content/drive/MyDrive/train_ds/Sample005/img005-004-edited.png"))

Data(x=[17, 2], edge_index=[2, 32])


In [21]:
import random

def generate_random(x,x1,x2):
    while True:
        random_number = random.randint(x1,x2)  # Generate a random number between 0 and 1
        if random_number != x:
            return random_number

def three_digit_num(x):
  if x < 10:
    return "00"+ str(x)
  else:
    return "0"+str(x)

In [22]:
import time




top_dir = "/content/drive/MyDrive/train_ds/Sample"

number_of_classes = 62
number_of_samples = 55
number_of_train_samples = 44
all_graphs = [[0]*(number_of_samples+1) for i in range(number_of_classes+1)]

count = 0
start_time = time.time()
for i in range(1,number_of_classes+1):
  for j in range(1,number_of_samples+1):
    if count %100 == 0 and count != 0:
      passed_time = time.time() - start_time
      print(f"time spent between itr {count} and itr {count-100}++++++==  {passed_time}")
      start_time = time.time()
    d = graph_making(top_dir+three_digit_num(i)+"/img"+three_digit_num(i)+"-"+three_digit_num(j)+"-edited.png")
    all_graphs[i][j] = d
    count +=1


time spent between itr 100 and itr 0++++++==  33.08451008796692
time spent between itr 200 and itr 100++++++==  75.07326292991638
time spent between itr 300 and itr 200++++++==  79.30572652816772
time spent between itr 400 and itr 300++++++==  77.55306935310364
time spent between itr 500 and itr 400++++++==  75.07497096061707
time spent between itr 600 and itr 500++++++==  77.5778419971466
time spent between itr 700 and itr 600++++++==  81.43349933624268
time spent between itr 800 and itr 700++++++==  84.08327627182007
time spent between itr 900 and itr 800++++++==  82.09567856788635
time spent between itr 1000 and itr 900++++++==  82.10971522331238
time spent between itr 1100 and itr 1000++++++==  73.34889912605286
time spent between itr 1200 and itr 1100++++++==  80.91939210891724
time spent between itr 1300 and itr 1200++++++==  84.92327928543091
time spent between itr 1400 and itr 1300++++++==  87.56163263320923
time spent between itr 1500 and itr 1400++++++==  88.74996280670166
ti

In [23]:



l1 = []
l2 = []
l3 = []

samples = 50000
start_time = time.time()
for i in range(samples):
  anchor_class = generate_random(-1,1,number_of_classes)
  other_class = generate_random(anchor_class,1,number_of_classes)
  anchor_sample = generate_random(-1,1,number_of_train_samples)
  pos_sample = generate_random(anchor_sample,1,number_of_train_samples)
  neg_sample = generate_random(-1,1,number_of_train_samples)
  l1.append(all_graphs[anchor_class][anchor_sample])
  l2.append(all_graphs[anchor_class][pos_sample])
  l3.append(all_graphs[other_class][neg_sample])


from torch_geometric.loader import DataLoader

train_loader1 = DataLoader(l1, batch_size=32, shuffle=False)
train_loader2 = DataLoader(l2,batch_size=32,shuffle = False)
trian_loader3 = DataLoader(l3,batch_size = 32, shuffle = False)






In [24]:
t_l1 = []
t_l2 = []
t_l3 = []

samples = 2500
start_time = time.time()
for i in range(samples):
  anchor_class = generate_random(-1,1,number_of_classes)
  other_class = generate_random(anchor_class,1,number_of_classes)
  anchor_sample = generate_random(-1,1,number_of_train_samples)
  pos_sample = generate_random(anchor_sample,1,number_of_train_samples)
  neg_sample = generate_random(-1,1,number_of_train_samples)
  l1.append(all_graphs[anchor_class][anchor_sample])
  l2.append(all_graphs[anchor_class][pos_sample])
  l3.append(all_graphs[other_class][neg_sample])


from torch_geometric.loader import DataLoader

test_loader1 = DataLoader(l1, batch_size=32, shuffle=False)
test_loader2 = DataLoader(l2,batch_size=32,shuffle = False)
test_loader3 = DataLoader(l3,batch_size = 32, shuffle = False)


In [25]:
import torch
torch.save(train_loader1,"/content/drive/MyDrive/tl1.pth")
torch.save(train_loader2,"/content/drive/MyDrive/tl2.pth")
torch.save(trian_loader3,"/content/drive/MyDrive/tl3.pth")
torch.save(test_loader1,"/content/drive/MyDrive/test_tl1.pth")
torch.save(test_loader2,"/content/drive/MyDrive/test_tl2.pth")
torch.save(test_loader3,"/content/drive/MyDrive/test_tl3.pth")